# RWKV RavenShift

The following is a mad experiment in running existing raven code, directly with the new token shift

## Basic Setup

In [10]:
# Lets download the required models
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/
!cd ../../../../model/ && wget -nc https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-3B-v12-Eng98%25-Other2%25-20230520-ctx4096.pth
# !cd ../../../../model/ && wget -nc https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-7B-v12-Eng98%25-Other2%25-20230521-ctx8192.pth
!ls -alh ../../../../model/

--2023-07-22 14:09:04--  https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-3B-v12-Eng98%25-Other2%25-20230520-ctx4096.pth
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.55, 13.33.33.110, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/41/55/4155c7aaff64e0f4b926df1a8fff201f8ee3653c39ba67b31e4973ae97828633/1eea1845acfe9729dfdaec66a8d1aeb91a1287d94bebbca5529c13c050540b33?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-4-Raven-3B-v12-Eng98%2525-Other2%2525-20230520-ctx4096.pth%3B+filename%3D%22RWKV-4-Raven-3B-v12-Eng98%25-Other2%25-20230520-ctx4096.pth%22%3B&Expires=1690265344&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MDI2NTM0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80MS81NS80MTU1YzdhYWZmNjRlMGY0YjkyNmRmMWE4ZmZmMjAxZjhlZTM2

In [3]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="RavenShift-Exp"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/rwkv5-tokenshift-experiment/notebook/experiment/tokenshift-exp/RavenShift
INFERENCE_DIR: /home/picocreator/rwkv-proj/rwkv5-tokenshift-experiment/RWKV-v4wavenet
TRAINER_DIR: /home/picocreator/rwkv-proj/rwkv5-tokenshift-experiment/RWKV-v4wavenet
PROJECT_DIR: /home/picocreator/rwkv-proj/rwkv5-tokenshift-experiment


## Mad Science

In [16]:
# Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/RWKV-4-Raven-3B-v12-Eng98%-Other2%-20230520-ctx4096.pth" "cuda" 500

[2023-07-22 14:21:48,088] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. The Chinese population was what it's made in the future.

Question:
What is the meaning of the word 'that'', the mea

In [13]:
# Lets do a memory eval 
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/RWKV-4-Raven-3B-v12-Eng98%-Other2%-20230520-ctx4096.pth" 

[2023-07-22 14:17:25,986] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 80.0% similarity, with 4 matched token, and 1 token mismatch
## Model validation for 10 tokens : 70.0% similarity, with 7 matched token, and 3 token mismatch
## Model validation for 15 tokens : 66.66666666666666% similarity, with 10 matched token, and 5 token mismatch
## Model v